<a href="https://colab.research.google.com/github/battuzz/torch_aot/blob/main/TorchAOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AOT Compilation of torch models

In [1]:
import torch

## Model definition

In [2]:
NUM_INPUTS = 5
NUM_OUTPUTS = 7
NUM_INDUCING_POINTS = 350

class ModelNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(NUM_INPUTS, 128)
        self.fc2 = torch.nn.Linear(128, 128)
        self.fc3 = torch.nn.Linear(128, NUM_OUTPUTS)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

def squared_distance(x1, x2):
    return (
        torch.sum(x1**2, dim=1, keepdim=True)
        + torch.sum(x2**2, dim=1)
        - 2 * torch.mm(x1, x2.t())
    )


def rbf_kernel(x1, x2, lengthscale=1.0):
    dist = squared_distance(x1 / lengthscale, x2 / lengthscale)
    return torch.exp(-0.5 * dist)

class ModelGPPosterior(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lengthscales = torch.nn.Parameter(torch.randn(NUM_INPUTS))
        self.inducing_points = torch.nn.Parameter(
        torch.randn(NUM_INDUCING_POINTS, NUM_INPUTS)
        )
        self.alpha = torch.nn.Parameter(torch.randn(NUM_INDUCING_POINTS, NUM_OUTPUTS))

    def forward(self, x):
        Kuf = rbf_kernel(x, self.inducing_points, self.lengthscales)
        mean = Kuf @ self.alpha
        return mean

In [3]:
def train_with_random_data(model):
    X = torch.randn(1000, NUM_INPUTS)
    y = torch.randn(1000, NUM_OUTPUTS)

    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    for epoch in range(30):
        optimizer.zero_grad()
        output = model(X)
        loss = torch.nn.functional.mse_loss(output, y)
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

In [4]:
model_nn = ModelNN()
model_gp = ModelGPPosterior()
train_with_random_data(model_nn)
train_with_random_data(model_gp)

Epoch 1, Loss: 1.0243384838104248
Epoch 2, Loss: 1.0232675075531006
Epoch 3, Loss: 1.0222439765930176
Epoch 4, Loss: 1.0212637186050415
Epoch 5, Loss: 1.0203267335891724
Epoch 6, Loss: 1.0194319486618042
Epoch 7, Loss: 1.0185813903808594
Epoch 8, Loss: 1.0177745819091797
Epoch 9, Loss: 1.017008900642395
Epoch 10, Loss: 1.0162802934646606
Epoch 11, Loss: 1.0155872106552124
Epoch 12, Loss: 1.0149301290512085
Epoch 13, Loss: 1.014306902885437
Epoch 14, Loss: 1.0137146711349487
Epoch 15, Loss: 1.0131503343582153
Epoch 16, Loss: 1.0126134157180786
Epoch 17, Loss: 1.0121012926101685
Epoch 18, Loss: 1.0116132497787476
Epoch 19, Loss: 1.0111464262008667
Epoch 20, Loss: 1.0106998682022095
Epoch 21, Loss: 1.0102720260620117
Epoch 22, Loss: 1.0098613500595093
Epoch 23, Loss: 1.0094648599624634
Epoch 24, Loss: 1.009081482887268
Epoch 25, Loss: 1.0087114572525024
Epoch 26, Loss: 1.00835120677948
Epoch 27, Loss: 1.007997989654541
Epoch 28, Loss: 1.0076525211334229
Epoch 29, Loss: 1.007313847541809
E

In [5]:
example_input = torch.randn((1, NUM_INPUTS))
model_nn.eval()

exported = torch.export.export(model_nn, (example_input,))
torch._inductor.aoti_compile_and_package(
    exported,
    package_path="model_nn.pt2",
)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


'model_nn.pt2'

In [6]:
!pip install cmake

In [7]:
cmake_contents = """cmake_minimum_required(VERSION 3.18 FATAL_ERROR)
project(aoti_example)

find_package(Torch REQUIRED)

add_executable(aoti_example inference.cpp)

target_link_libraries(aoti_example "${TORCH_LIBRARIES}")
set_property(TARGET aoti_example PROPERTY CXX_STANDARD 17)
"""
with open('CmakeLists.txt', 'w') as f:
    f.write(cmake_contents)

In [8]:
build_contents = """export CMAKE_PREFIX_PATH=/home/abattistello/.venv/lib/python3.11/site-packages/torch/share/cmake
export TORCHINDUCTOR_FREEZING=1


rm -rf build
mkdir build
cmake -B build .
cmake --build build --config Release
"""
with open('build.sh', 'w') as f:
    f.write(build_contents)

In [9]:
!chmod +x ./build.sh

In [ ]:
cpp_content = """#include <iostream>
#include <vector>
#include <chrono>
#include <fstream>

#include <torch/torch.h>
#include <torch/csrc/inductor/aoti_package/model_package_loader.h>

using namespace std::chrono;

int main(int argc, char* argv[]) {

    if (argc < 3) {
        std::cerr << "Usage: " << argv[0] << " <model.pt2> <inputs.txt>" << std::endl;
        return 1;
    }

    // Load input
    std::ifstream input_file{argv[2]};
    if (!input_file) {
        std::cerr << "Error opening input file: " << argv[2] << std::endl;
        return 1;
    }
    int num_dims {};
    input_file >> num_dims;

    std::vector<int64_t> input_dims{};
    for (int i = 0; i < num_dims; ++i) {
        int64_t dim_size;
        input_file >> dim_size;
        input_dims.push_back(dim_size);
    }

    input_file.close();

    // auto arrayRef = c10::makeArrayRef(input_dims);
    torch::Tensor input = torch::randn(input_dims, torch::dtype(torch::kFloat64));
    std::vector<torch::Tensor> inputs { input };

    c10::InferenceMode mode;
    torch::inductor::AOTIModelPackageLoader loader(argv[1], "model", false);


    // std::vector<torch::Tensor> inputs = {torch::randn({1, 2}, torch::dtype(torch::kFloat64))};

    // Warmup
    std::vector<torch::Tensor> outputs;
    for (int i = 0; i < 1000; i++) {
        outputs = loader.run(inputs);
    }

    // Benchmark
    auto start_time = std::chrono::high_resolution_clock::now();
    for (int i = 0; i < 1000; i++) {
        outputs = loader.run(inputs);
    }
    auto end_time = std::chrono::high_resolution_clock::now();

    auto elapsed = duration_cast<microseconds>(end_time - start_time);
    std::cout << "Average inference time over 1000 runs: "
              << (elapsed.count() / 1000) << " us" << std::endl;

    return 0;
}
"""

In [ ]:
!./build.sh

./build.sh: line 3: $'\r': command not found
./build.sh: line 4: $'\r': command not found
CMake Warning:
  Ignoring extra path from command line:

"


CMake Error at CMakeLists.txt:4 (find_package):
  By not providing "FindTorch.cmake" in CMAKE_MODULE_PATH this project has
  asked CMake to find a package configuration file provided by "Torch", but
  CMake did not find one.

  Could not find a package configuration file provided by "Torch" with any of
  the following names:

    TorchConfig.cmake
    torch-config.cmake

  Add the installation prefix of "Torch" to CMAKE_PREFIX_PATH or set
  "Torch_DIR" to a directory containing one of the above files.  If "Torch"
  provides a separate development package or SDK, be sure it has been
  installed.


-- Configuring incomplete, errors occurred!
gmake: Makefile: No such file or directory
gmake: *** No rule to make target 'Makefile'.  Stop.
./build.sh: line 9: $'\r': command not found
